In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

stocks = {
    'AAPL': pd.read_csv('aapl.us.csv'),
    'AMZN': pd.read_csv('amzn.us.csv'),
    'GOOGL': pd.read_csv('googl.us.csv'),
    'MSFT': pd.read_csv('msft.us.csv'),
    'TSLA': pd.read_csv('tsla.us.csv')
}

In [2]:
for ticker, df in stocks.items():
    print(ticker)
    print(df.shape)
    print(list(df.columns))
    print(f"{df['Date'].min()}  {df['Date'].max()}")
    print(df.isnull().sum().sum())

AAPL
(8364, 7)
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
1984-09-07 to 2017-11-10
0
AMZN
(5153, 7)
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
1997-05-16 to 2017-11-10
0
GOOGL
(3333, 7)
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
2004-08-19 to 2017-11-10
0
MSFT
(7983, 7)
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
1986-03-13 to 2017-11-10
0
TSLA
(1858, 7)
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
2010-06-28 to 2017-11-10
0


In [5]:
processed_stocks = {}

for ticker, df in stocks.items():
    stock_df = df.copy()
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
    stock_df = stock_df.sort_values('Date').reset_index(drop=True)
    stock_df = stock_df.dropna(subset=['Close'])
    stock_df['Ticker'] = ticker
    stock_df['Daily_Return'] = stock_df['Close'].pct_change()
    stock_df['MA_7'] = stock_df['Close'].rolling(window=7).mean()
    stock_df['MA_30'] = stock_df['Close'].rolling(window=30).mean()
    stock_df['Volatility_30'] = stock_df['Daily_Return'].rolling(window=30).std()
    processed_stocks[ticker] = stock_df

combined_df = pd.concat(processed_stocks.values(), ignore_index=True)

print(combined_df.head())

        Date     Open     High      Low    Close    Volume  OpenInt Ticker  \
0 1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030        0   AAPL   
1 1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532        0   AAPL   
2 1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199        0   AAPL   
3 1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801        0   AAPL   
4 1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062        0   AAPL   

   Daily_Return  MA_7  MA_30  Volatility_30  
0           NaN   NaN    NaN            NaN  
1     -0.005992   NaN    NaN            NaN  
2      0.018228   NaN    NaN            NaN  
3     -0.029929   NaN    NaN            NaN  
4      0.055481   NaN    NaN            NaN  


In [7]:
ten_years_ago = datetime.now() - timedelta(days=365*10)
recent_data = combined_df[combined_df['Date'] >= ten_years_ago].copy()

In [8]:
multi_df = recent_data.set_index(['Ticker', 'Date'])

In [9]:
returns_summary = recent_data.groupby('Ticker')['Daily_Return'].describe()
print(returns_summary)

        count      mean       std       min       25%       50%       75%  \
Ticker                                                                      
AAPL    615.0  0.000677  0.014421 -0.065703 -0.005499  0.000553  0.007881   
AMZN    615.0  0.001726  0.017345 -0.078288 -0.005818  0.001168  0.009679   
GOOGL   615.0  0.001145  0.014327 -0.055521 -0.004901  0.001251  0.007355   
MSFT    615.0  0.001155  0.013431 -0.071733 -0.004702  0.000695  0.007110   
TSLA    615.0  0.000617  0.024413 -0.104503 -0.011918  0.000871  0.014393   

             max  
Ticker            
AAPL    0.064964  
AMZN    0.132164  
GOOGL   0.162584  
MSFT    0.100771  
TSLA    0.111735  


In [10]:

print(recent_data.groupby('Ticker')['Volatility_30'].describe())


        count      mean       std       min       25%       50%       75%  \
Ticker                                                                      
AAPL    615.0  0.013605  0.004582  0.005085  0.010807  0.012721  0.015768   
AMZN    615.0  0.015443  0.006805  0.005160  0.010540  0.012557  0.019950   
GOOGL   615.0  0.012956  0.006134  0.004231  0.009032  0.011436  0.014354   
MSFT    615.0  0.012505  0.004879  0.004159  0.008628  0.011621  0.015880   
TSLA    615.0  0.023433  0.005798  0.011555  0.019719  0.022723  0.026530   

             max  
Ticker            
AAPL    0.027176  
AMZN    0.035086  
GOOGL   0.038067  
MSFT    0.023590  
TSLA    0.040158  


ACCORDING TO VOLATILTY- TSLA < AMZN < AAPL < GOOGL < MSFT

ACCORDING TO DAILY RETURN MEAN  - AMZN> MSFT> GOOGL > AAPL > TSLA



In [11]:
print(recent_data.groupby('Ticker')['Close'].describe())

        count        mean         std      min      25%      50%      75%  \
Ticker                                                                      
AAPL    615.0  120.039850   22.057446   87.979  104.525  112.910  140.415   
AMZN    615.0  749.568545  174.121633  423.500  598.595  759.220  893.195   
GOOGL   615.0  799.128436  114.129939  540.040  729.085  788.480  856.865   
MSFT    615.0   57.516237   10.645086   38.462   49.860   55.903   64.757   
TSLA    615.0  254.174441   55.215275  143.670  213.745  238.630  296.150   

            max  
Ticker           
AAPL     175.61  
AMZN    1132.88  
GOOGL   1058.29  
MSFT      84.56  
TSLA     385.00  


In [13]:
recent_data.to_csv('processed_stocks_combined.csv', index=False)
multi_df.to_csv('processed_stocks_multiindex.csv')

In [14]:
for ticker in ['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA']:
    ticker_data = recent_data[recent_data['Ticker'] == ticker]
    ticker_data.to_csv(f'processed_{ticker.lower()}.csv', index=False)

print(len(recent_data))
print(recent_data['Ticker'].nunique())
print(recent_data['Ticker'].value_counts())

print(recent_data.head(10))
print(list(recent_data.columns))
print(multi_df.head())

3075
5
Ticker
AAPL     615
AMZN     615
GOOGL    615
MSFT     615
TSLA     615
Name: count, dtype: int64
           Date    Open    High     Low   Close    Volume  OpenInt Ticker  \
7749 2015-06-08  123.02  123.32  121.03  121.97  55195489        0   AAPL   
7750 2015-06-09  120.91  122.22  119.88  121.61  58732198        0   AAPL   
7751 2015-06-10  122.09  123.45  122.00  123.00  40948345        0   AAPL   
7752 2015-06-11  123.29  124.22  122.60  122.71  37084550        0   AAPL   
7753 2015-06-12  122.32  122.46  121.31  121.37  38579953        0   AAPL   
7754 2015-06-15  120.34  121.43  119.97  121.12  46094023        0   AAPL   
7755 2015-06-16  121.22  122.00  120.59  121.78  33001274        0   AAPL   
7756 2015-06-17  121.89  122.03  120.95  121.49  34493316        0   AAPL   
7757 2015-06-18  121.43  122.44  121.42  122.03  37101639        0   AAPL   
7758 2015-06-19  121.92  121.98  120.62  120.81  57335314        0   AAPL   

      Daily_Return        MA_7       MA_30  Vol

In [15]:
recent_data['Date'] = pd.to_datetime(recent_data['Date'])

recent_data['Year'] = recent_data['Date'].dt.year
recent_data['Month'] = recent_data['Date'].dt.month

In [16]:
monthly_volatility = recent_data.groupby(['Ticker', 'Year', 'Month'])['Daily_Return'].std()

max_volatility_idx = monthly_volatility.idxmax()
max_volatility_value = monthly_volatility.max()

ticker, year, month = max_volatility_idx

print(ticker)
print(year)
print(month)
print(max_volatility_value)

TSLA
2016
2
0.046379164019557564


TSLA THE MAX VOLATILE MONTH AS EXPECTED :P